In [ ]:
import numpy as np
import sys, os
from pathlib import Path
import pandas as pd
import copy
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors
import math
import datetime as dt
from sklearn.metrics import r2_score
import plotly.colors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.interpolate import griddata
from sklearn.linear_model import LinearRegression
from scipy import odr
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import logging
logger = logging.getLogger()
logging.basicConfig(format='%(asctime)s.%(msecs)03d %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
level = logging.getLevelName('INFO')
logger.setLevel(level)

# ----- Internal Dependencies -------#
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append("../..")
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from General import FileRead
from General import AccuracyMetrics
from General.GeneralFunctions import get_add_to_dict
from General import TableManipulations
from FeatureGeneration import GeneralFeatures
from General import FileWrite

from Plot.PdfHelper import PdfHelper
from Plot.PlotMaker import PlotMaker
# from Plot import Voltage_Plots
from Plot import PlotFunctions
# from Plot import AdHocPlots
# from Plot import Sim_Plots

# widget and notebook stuff
from ipywidgets import interact, fixed
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
import pathlib
pathlib.WindowsPath = pathlib.PosixPath

In [ ]:
def simple_linear_model(B, x):
    return B[0]+B[1]*x

def retro_cal_gluc_by_ref(biolinq_df,ref_df,col_to_match = 'UTC_Time',tol_to_match='7.5min',gluc_col_name='biolinq',ref_col_name='acck'):
    biolinq_ref_df = pd.merge_asof(left=ref_df.dropna(subset=[col_to_match]).sort_values(by=col_to_match), right=biolinq_df.dropna(subset=[col_to_match]).sort_values(by=col_to_match),
                                on=col_to_match, direction='nearest', tolerance=pd.Timedelta(tol_to_match))
    biolinq_ref_df = biolinq_ref_df.dropna(subset=[ref_col_name])
    biolinq_ref_df = biolinq_ref_df.dropna(subset=[gluc_col_name])
    slope,intercept,rmse = fit_odr_gluc_ref(biolinq_ref_df,gluc_col_name,ref_col_name)
    if (biolinq_ref_df.shape[0]>=10) and (biolinq_ref_df[ref_col_name].max()-biolinq_ref_df[ref_col_name].min()>=30) and (intercept>=-60) and (intercept<=60) and (slope>=0.4) and (slope<=1.6):
        biolinq_df[f'{gluc_col_name}_adj']=biolinq_df[gluc_col_name].subtract(intercept).divide(slope)
    else:
        biolinq_df[f'{gluc_col_name}_adj']=biolinq_df[gluc_col_name]
    return slope,intercept,rmse

def fit_odr_gluc_ref(biolinq_ref_df,gluc_col_name,ref_col_name):
    # Orthogonal regression
    orthogonal_model = odr.Model(simple_linear_model)
    current_data = odr.Data(biolinq_ref_df[ref_col_name],biolinq_ref_df[gluc_col_name])
    orthogonal_reg = odr.ODR(current_data,orthogonal_model,beta0=[0.0,1.0])
    orthogonal_reg_outcome = orthogonal_reg.run()
    intercept = orthogonal_reg_outcome.beta[0]
    slope = orthogonal_reg_outcome.beta[1]
    rmse = np.sqrt(orthogonal_reg_outcome.res_var)
    return slope,intercept,rmse


# Settings

In [ ]:
#load processed data
# data_folder_parent = '/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Documents - Clinical Data Analysis/Data/Aggregate/Isabella_v01'
# data_dict={
#     # 'eBlinq14':Path(data_folder_parent,'eBlinq14'),
#     # 'eBlinq15':Path(data_folder_parent,'eBlinq15'),
#     # 'eBlinq16':Path(data_folder_parent,'eBlinq16'),
#     # 'eBlinq17':Path(data_folder_parent,'eBlinq17'),
#     # 'eBlinq18':Path(data_folder_parent,'eBlinq18'),
#     # 'eBlinq19a':Path(data_folder_parent,'eBlinq19a'),
#     # 'eBlinq19b':Path(data_folder_parent,'eBlinq19b'),
#     'eBlinq19c':Path(data_folder_parent,'eBlinq19c'),
#     # 'eBlinq20':Path(data_folder_parent,'eBlinq20'),
#     # 'eBlinq22':Path(data_folder_parent,'eBlinq22'),
#     # 'eBlinq23':Path(data_folder_parent,'eBlinq23'),
#     # 'eBlinq25':Path(data_folder_parent,'eBlinq25'),
#     # 'eBlinqRingOverlay&NiMNA':Path(data_folder_parent,'eBlinqRingOverlay&NiMNA'),
#     # 'iBlinq16':Path(data_folder_parent,'iBlinq16'),
#     # 'iBlinqAcet':Path(data_folder_parent,'iBlinqAcet'),
#     # 'iBlinqAdhesive':Path(data_folder_parent,'iBlinqAdhesive'),
#     # 'iBlinqFW231':Path(data_folder_parent,'iBlinqFW231'),
#     # 'iBlinqOverlay':Path(data_folder_parent,'iBlinqOverlay'),
#     # 'Pre-Piv-2-training':Path(data_folder_parent,'Pre-Piv-2-training'),
# }

data_folder_parent = '/Volumes/Shared/Algo/Algorithm Output/new_run'
data_dict={
    'Model_May':Path(data_folder_parent,'eBlinqMayStudy')
}

In [ ]:
#Consolidated MFG data
# mfg_data=pd.read_excel(r'C:\Users\elizabeth\OneDrive - Biolinq Inc\Data\Aggregate\eblinqiblinq 14-23\training\BLINQ TRENDING MONEY YIELD TABLE no formulas.xlsx')
# mfg_data=pd.read_excel(r'/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Documents - Clinical Data Analysis/Data/Aggregate/Algo Validation/Algo Validation - Sensor Map.xlsx',sheet_name='All')

#List of features to gather from sample df
cols_all=pd.read_excel('/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Gen 1/Algorithm Development/Gen 1 Model Optimization/features_for_optimization.xlsx')['feature']

#Columns to keep from sensor map
cols_sensor_map=['Sensor_Id']

#List of 300 Hz features
cols_300=[
'real1_300_alignment',
'real3_300_alignment',
'real4_300_alignment',
'imag1_300_alignment',
'imag3_300_alignment',
'imag4_300_alignment',
'deg1_300_featureGeneration',
'mag1_300_featureGeneration',
'deg3_300_featureGeneration',
'mag3_300_featureGeneration',
'deg4_300_featureGeneration',
'mag4_300_featureGeneration',
'alt_nyquistlength_cur1_featureGeneration',
'alt_nyquistlength_cur3_featureGeneration',
'alt_nyquistlength_cur4_featureGeneration',
'alt_nyquistslope_cur1_featureGeneration',
'alt_nyquistslope_cur3_featureGeneration',
'alt_nyquistslope_cur4_featureGeneration',
'diff_ewm_average_8hr_deg1_300_featureGeneration',
'diff_ewm_average_8hr_deg3_300_featureGeneration',
'diff_ewm_average_8hr_deg4_300_featureGeneration',
'diff_ewm_average_8hr_mag1_300_featureGeneration',
'diff_ewm_average_8hr_mag3_300_featureGeneration',
'diff_ewm_average_8hr_mag4_300_featureGeneration',
'estimate_standard_deviation_12hr_deg1_300_featureGeneration',
'estimate_standard_deviation_12hr_deg3_300_featureGeneration',
'estimate_standard_deviation_12hr_deg4_300_featureGeneration',
'estimate_standard_deviation_12hr_mag1_300_featureGeneration',
'estimate_standard_deviation_12hr_mag3_300_featureGeneration',
'estimate_standard_deviation_12hr_mag4_300_featureGeneration',
'estimate_standard_deviation_24hr_deg1_300_featureGeneration',
'estimate_standard_deviation_24hr_deg3_300_featureGeneration',
'estimate_standard_deviation_24hr_deg4_300_featureGeneration',
'estimate_standard_deviation_24hr_mag1_300_featureGeneration',
'estimate_standard_deviation_24hr_mag3_300_featureGeneration',
'estimate_standard_deviation_24hr_mag4_300_featureGeneration',
'ewm_average_8hr_deg1_300_featureGeneration',
'ewm_average_8hr_deg3_300_featureGeneration',
'ewm_average_8hr_deg4_300_featureGeneration',
'ewm_average_8hr_mag1_300_featureGeneration',
'ewm_average_8hr_mag3_300_featureGeneration',
'ewm_average_8hr_mag4_300_featureGeneration',
'ratio_mag1_10000_to_mag1_300_featureGeneration',
'ratio_mag3_10000_to_mag3_300_featureGeneration',
'ratio_mag4_10000_to_mag4_300_featureGeneration',
]

In [ ]:
#Whether to keep 300Hz features
keep_300=False

#Whether to merge mfg data
keep_mfg=False

#List of reference CSVs to create
ref_to_use = 'cgm_adj'

#Output folder
file_path = '/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Gen 1/Algorithm Development/Gen 1 Model Optimization'
file_name = "agg_sample_df_ref_May_Study_mfg.csv"
#Column to merge on
time_to_match = 'UTC_Time'

#Specify glucose channels
ch_str = ["1","3","4"]
gluc_prefix="model_output_s"
gluc_suffix="_calculateGlucose"

cgm_cols_needed = ['UTC_Time','cgm','cgm_adj','cgm_name']
acck_cols_needed = ['UTC_Time','acck']
ysi_cols_needed = ['UTC_Time','ysi']

# rolling logics
rolling_window = 2880 #24 hours
rolling_min_size_calc = 100
corr_tracking_thresh = 0.8

# Run

In [ ]:
#only keep specified columns
if not keep_300:
    cols_all=cols_all[~cols_all.isin(cols_300)].to_list()
if False: # keep_mfg:
    cols_all=pd.concat([cols_all,pd.Series(mfg_data.columns)]).to_list()

cols_all=cols_all+cols_sensor_map
cols_all.append("Study")
cols_all.append("Ref")
cols_all.append("RefType")
cols_all=list(set(cols_all))
cols_all=sorted(cols_all)

In [ ]:
### RUN CONSOLIDATION ###

#Output file name
file_path_name=os.path.join(file_path,file_name)

#check if data consolidation file already exists
if os.path.isfile(file_path_name):
    raise FileExistsError("File already exists. "+file_path_name)

#Save the header for the first sensor in the file
head=False

#load data study by study
for i,ds in enumerate(data_dict.keys()):
    logger.info(f'Processing {ds}')
    #Load dataset
    loaded_data=FileRead.load_pickle('alg_out.zip', data_dict[ds], as_dict=True)
    sensor_map_df=loaded_data['sensor_map_df']

    #Sensor for loop
    for j,sensor in enumerate(loaded_data['all_sensor_data'].keys()):
        logger.info(f'Aggregating {sensor}')
        #Load data and create UTC Time column
        current_sensor_data = loaded_data['all_sensor_data'][sensor]
        sample_df_exist = False
        if 'sample_df' in current_sensor_data.keys():
            df = current_sensor_data['sample_df']
            sample_df_exist = True
        else:
            continue

        #Populate study column
        df['Study']=ds

        #Populate columns from sensor map
        for c in cols_sensor_map:
            df[c]=sensor_map_df[sensor_map_df['Sensor_Id']==sensor][c].values[0]

        #Merge MFG data
        if keep_mfg:
            # df=pd.merge(left=df,right=mfg_data,how='left',on='SN')
            df=pd.merge(left=df.drop(columns=['Study']),right=mfg_data.rename(columns={'Grouping':'Study'}),how='left',on='Sensor_Id')
            
        # pull all reference dataframe
        acck_exist = False
        ysi_exist = False
        Libre_exist = False
        Dexcom_exist = False
        cgm_exist = False            
        if 'acck' in current_sensor_data.keys():
            acck_exist = True
            acck_df = current_sensor_data['acck'].copy()
        if 'ysi' in current_sensor_data.keys():
            ysi_exist = True
            ysi_df = current_sensor_data['ysi'].copy()
        if 'Libre' in current_sensor_data.keys():
            Libre_exist = True
            Libre_df = current_sensor_data['Libre'].copy()
        if 'Dexcom' in current_sensor_data.keys():
            Dexcom_exist = True
            Dexcom_df = current_sensor_data['Dexcom'].copy()            

        #If reference is adjusted, merge data and perform orthogonal regression fit. 
        #Protect from too little data, too little glucose range, too large of slopes, and too large of intercepts
        cgm_df = pd.DataFrame()
        
        if acck_exist and not acck_df.empty:
            if Libre_exist and not Libre_df.empty:
                _,_,_ = retro_cal_gluc_by_ref(Libre_df,acck_df,gluc_col_name='Libre')              
                cgm_df = Libre_df
                cgm_df.rename(columns={'Libre':'cgm'},inplace=True)
                cgm_df.rename(columns={'Libre_adj': 'cgm_adj'},inplace=True)
                cgm_df['cgm_name'] = 1
                cgm_exist = True
            elif Dexcom_exist and not Dexcom_df.empty:
                _,_,_ = retro_cal_gluc_by_ref(Dexcom_df,acck_df,gluc_col_name='Dexcom')              
                cgm_df = Dexcom_df
                cgm_df.rename(columns={'Dexcom':'cgm'},inplace=True)
                cgm_df.rename(columns={'Dexcom_adj': 'cgm_adj'},inplace=True)
                cgm_df['cgm_name'] = 2       
                cgm_exist = True

        if  cgm_exist and not cgm_df.empty and all(e in cgm_df.columns for e in cgm_cols_needed):
            df_cgm_paired = pd.merge_asof(left=df.dropna(subset=[time_to_match]).sort_values(by=time_to_match),right=cgm_df[cgm_cols_needed].dropna(subset=[time_to_match]).sort_values(by=time_to_match),on=time_to_match,direction='nearest',tolerance=pd.Timedelta('5min'))
            for current_ch_str in ch_str:
                current_col = gluc_prefix+current_ch_str+gluc_suffix
                df_cgm_paired_roll_corr = df_cgm_paired[[current_col,'cgm_adj']].rolling(rolling_window,min_periods=rolling_min_size_calc,center=True).corr(pairwise=True)
                df_cgm_paired_roll_corr.index.names = ['index','measurement']
                df_cgm_paired[f'retro_roll_corr_s{current_ch_str}'] = df_cgm_paired_roll_corr.query(f'measurement=="{current_col}"').reset_index()['cgm_adj']
                roll_ols_model= RollingOLS.from_formula(f'{current_col} ~ cgm_adj',data=df_cgm_paired,window=rolling_window,min_nobs=rolling_min_size_calc,missing='drop',expanding=True)
                fit_result = roll_ols_model.fit()
                df_cgm_paired[f'retro_roll_slope_s{current_ch_str}'] = fit_result.params['cgm_adj']
                df_cgm_paired[f'retro_roll_intercept_s{current_ch_str}'] = fit_result.params['Intercept']
                df_cgm_paired[f'retro_roll_rmse_s{current_ch_str}'] = fit_result.mse_resid.apply(np.sqrt)
                df_cgm_paired[f'retro_roll_{current_col}'] = df_cgm_paired[current_col].sub(df_cgm_paired[f'retro_roll_intercept_s{current_ch_str}']).div(df_cgm_paired[f'retro_roll_slope_s{current_ch_str}'])
                idx_tracking = df_cgm_paired[f'retro_roll_corr_s{current_ch_str}'].ge(corr_tracking_thresh)
                df_cgm_paired_copy = df_cgm_paired.loc[idx_tracking].dropna(subset=[current_col,'cgm_adj']).copy()
                slope,intercept,rmse = fit_odr_gluc_ref(df_cgm_paired_copy,current_col,'cgm_adj')                
                df_cgm_paired[f'retro_slope_s{current_ch_str}'] = slope
                df_cgm_paired[f'retro_intercept_s{current_ch_str}'] = intercept
                df_cgm_paired[f'retro_rmse_s{current_ch_str}'] = rmse
                df_cgm_paired[f'retro_{current_col}'] = df_cgm_paired[current_col].sub(df_cgm_paired[f'retro_intercept_s{current_ch_str}']).div(df_cgm_paired[f'retro_slope_s{current_ch_str}'])
        else:
            df_cgm_paired = df

        if  acck_exist and not acck_df.empty and all(e in acck_df.columns for e in acck_cols_needed):
            df_cgm_acck_paired = pd.merge_asof(left=df_cgm_paired.dropna(subset=[time_to_match]).sort_values(by=time_to_match),right=acck_df[acck_cols_needed].dropna(subset=[time_to_match]).sort_values(by=time_to_match),on=time_to_match,direction='nearest',tolerance=pd.Timedelta('5min'))
        else:
            df_cgm_acck_paired = df_cgm_paired

        if  ysi_exist and not ysi_df.empty and all(e in ysi_df.columns for e in ysi_cols_needed):
            df_cgm_acck_ysi_paired = pd.merge_asof(left=df_cgm_acck_paired.dropna(subset=[time_to_match]).sort_values(by=time_to_match),right=ysi_df[ysi_cols_needed].dropna(subset=[time_to_match]).sort_values(by=time_to_match),on=time_to_match,direction='nearest',tolerance=pd.Timedelta('5min'))
        else:
            df_cgm_acck_ysi_paired = df_cgm_acck_paired

        #Clean paired data
        for current_ch_str in ch_str:
            current_col = gluc_prefix+current_ch_str+gluc_suffix
            if current_col not in df_cgm_acck_ysi_paired.columns:
                continue
            df_cgm_acck_ysi_paired=df_cgm_acck_ysi_paired.dropna(subset=[current_col])                                        
        if df_cgm_acck_ysi_paired.empty:
            continue

        #Fill in desired columns with NaN
        for col in cols_all:
            if col not in df_cgm_acck_ysi_paired.columns:
                df_cgm_acck_ysi_paired[col]=np.nan
        
        #Save to CSV
        if head==False:
            df_cgm_acck_ysi_paired[cols_all].to_csv(file_path_name,mode='a',float_format='%g',index=False,header=True)
            head=True
        else:
            df_cgm_acck_ysi_paired[cols_all].to_csv(file_path_name,mode='a',float_format='%g',index=False,header=False) 